In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pdb
print_tensor = lambda n, x: print(n, type(x), x.shape, x.min(), x.max())
class WindowAttentionTest(nn.Module):
    """ Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        print_tensor('pos_bias_tb', self.relative_position_bias_table)
        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        
        print_tensor('coords', coords)
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        print_tensor('relative coords', relative_coords)

        pdb.set_trace()
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0

        relative_coords[:, :, 1] += self.window_size[1] - 1

        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1

        
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """ Forward function.

        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        


In [ ]:
windatt = WindowAttentionTest(32, (3, 3), 3)

In [7]:

class WindowAttention3d(nn.Module):
    """ Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window. example [7, 7, 7]
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww, Wd
        self.window_volume = np.prod(self.window_size)
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1) * (2 * window_size[2] - 1), num_heads))  
            # 2*Wh-1 * 2*Ww-1 * 2*Wd-1 nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords_d = torch.arange(self.window_size[2])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w, coords_d]))  # 3, Wh, Ww, Wd
        coords_flatten = torch.flatten(coords, 1)  # 3, Wh*Ww*Wd
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 3, Wh*Ww*Wd, Wh*Ww*Wd

        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww*Wd, Wh*Ww*Wd, 3
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 2] += self.window_size[2] - 1

        relative_coords[:, :, 0] *= (2 * self.window_size[1] - 1) * (2 * self.window_size[2] - 1) #TODO: how to adapt here 
        relative_coords[:, :, 1] *= (2 * self.window_size[2] - 1)
        
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww*Wd, Wh*Ww*Wd
        self.register_buffer("relative_position_index", relative_position_index)
        
        pdb.set_trace()
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout3d(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout3d(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1) # 

    def forward(self, x, mask=None):
        """ Forward function.

        Args:
            x: input features with shape of (num_windows*B, N, C) #  # x_windows: nW*B, window_size^3, C
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None # attn_mask: nW, window_size^3, window_size^3;
        """
        B_, N, C = x.shape # 

In [ ]:
windatt3d = WindowAttention3d(32, (2, 2, 2), 1)

> <ipython-input-7-4b19298a9d19>(50)__init__()
     48 
     49         pdb.set_trace()
---> 50         self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
     51         self.attn_drop = nn.Dropout3d(attn_drop)
     52         self.proj = nn.Linear(dim, dim)

ipdb> relative_position_index
tensor([[13, 12, 10,  9,  4,  3,  1,  0],
        [14, 13, 11, 10,  5,  4,  2,  1],
        [16, 15, 13, 12,  7,  6,  4,  3],
        [17, 16, 14, 13,  8,  7,  5,  4],
        [22, 21, 19, 18, 13, 12, 10,  9],
        [23, 22, 20, 19, 14, 13, 11, 10],
        [25, 24, 22, 21, 16, 15, 13, 12],
        [26, 25, 23, 22, 17, 16, 14, 13]])
ipdb> relative_coords[:, :, 0]
tensor([[ 9,  9,  9,  9,  0,  0,  0,  0],
        [ 9,  9,  9,  9,  0,  0,  0,  0],
        [ 9,  9,  9,  9,  0,  0,  0,  0],
        [ 9,  9,  9,  9,  0,  0,  0,  0],
        [18, 18, 18, 18,  9,  9,  9,  9],
        [18, 18, 18, 18,  9,  9,  9,  9],
        [18, 18, 18, 18,  9,  9,  9,  9],
        [18, 18, 18, 18,  9,  9,  9,  9]])
ipdb>